In [1]:
import pyspark
from pyspark.sql import SparkSession

# instantiate a Spark session, an object that we use to interact with Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
# read the zone data
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [3]:
# read the fhv_tripdata_2021-01.csv.gz 
df_fhv = spark.read \
    .option("header", "true") \
    .option("inferSchema", True) \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [4]:
df_fhv.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [5]:
df_fhv.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True)])

Keep only the first 1000 rows from the CSV

In [6]:
# Keep only the first 1000 rows from the CSV
# # BASH ONLY
# !head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

# first unzip the .GZ file with 7zip in Windows Explorer

# https://www.reddit.com/r/excel/comments/99njm0/how_do_i_get_the_first_1000_rows_from_a_6_gb_csv/
!powershell -command "& {get-content fhvhv_tripdata_2021-01.csv|select-object -first 10}" > head.csv

. : File C:\Users\nimz\Documents\WindowsPowerShell\profile.ps1 cannot be loaded. The file 
C:\Users\nimz\Documents\WindowsPowerShell\profile.ps1 is not digitally signed. You cannot run this script on the 
current system. For more information about running scripts and setting execution policy, see about_Execution_Policies 
at https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:1 char:3
+ . 'C:\Users\nimz\Documents\WindowsPowerShell\profile.ps1'
+   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess


Pandas should read in data types very accurately

In [7]:
import pandas as pd

In [8]:
df_pandas = pd.read_csv('head.csv')

In [9]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

**Turn this pandas dataframe into a Spark dataframe**

In [10]:
# create a Spark dataframe from the pandas dataframe and view the schema
spark.createDataFrame(df_pandas).show()

C:\spark\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\spark\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [11]:
# create a Spark dataframe from the pandas dataframe and view the schema
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

We want to change those `LongType` data types, which are less memory-efficient (use 8 bytes instead of the 4 bytes used by IntegerType), into Integers

It is also possible to observe that `pickup_datetime` and `dropoff_datetime` were both interpreted as strings

We want to achieve better performance and intepret all fields correctly, so we can specify the expected schema

In [12]:
from pyspark.sql import types

In [13]:
# specify expected schema, making sure the columsn can be nullable (that final "True" Boolean value)
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [14]:
# read in the dataframe with the expected schema
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [15]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

Right now our `fhvhv_tripdata_2021-01.csv` file is quite large

We currently have a **Spark Cluster**, inside of which are a bunch of **executors** (machines doing computational work)
- Let's say we have a bunch of files in our GCS Bucket containing a bunch of files
- Each of these files will be pulled to an executor
    - If we have more executors than files, Spark waits until one executor is done with a file and then that executor will grab one of the leftover files
- Say there's only 1 large file and 6 executors
    - One executor will grab this file, and the other 5 will be idle
    - This is not efficient
    - We'd rather have a bunch of smaller files
- We can break up the large file in Spark via **partitions**

In [16]:
# convert the large "file" into 24 partitions
df = df.repartition(24)

In [18]:
# save the dataframe (via its partitions)
df.write.parquet('fhvhv/2021/01/')

**You will be able to see all 24 partitions as files in the `fhvhv/2021/01/`dir**

**Can also see a `parquet` job at http://localhost:4040/jobs/ that we can click into and view more information such as DAGs**

In [21]:
!dir fhvhv\2021\01

 Volume in drive C has no label.
 Volume Serial Number is 08A3-CF2D

 Directory of C:\Users\nimz\Documents\de_zoomcamp\week5_batch_processing\fhvhv\2021\01

05/09/2023  08:47 PM    <DIR>          .
05/09/2023  08:47 PM    <DIR>          ..
05/09/2023  08:47 PM            71,484 .part-00000-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-c000.snappy.parquet.crc
05/09/2023  08:47 PM            71,468 .part-00001-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-c000.snappy.parquet.crc
05/09/2023  08:47 PM            71,476 .part-00002-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-c000.snappy.parquet.crc
05/09/2023  08:47 PM            71,480 .part-00003-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-c000.snappy.parquet.crc
05/09/2023  08:47 PM            71,480 .part-00004-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-c000.snappy.parquet.crc
05/09/2023  08:47 PM            71,484 .part-00005-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-c000.snappy.parquet.crc
05/09/2023  08:47 PM            71,444 .part-00006-1fc571d4-0079-41cb-ad93-7b7dd5e2db6c-

In [22]:
# read the partitioned files back into a Spark dataframe
df = spark.read.parquet('fhvhv/2021/01/')

In [23]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)

